In [ ]:
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
from sklearn.preprocessing import normalize
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

from qiskit import transpile
from qiskit_aer import AerSimulator
from qiskit.circuit.library import DiagonalGate, Initialize
from qiskit.quantum_info import Statevector
from qiskit.circuit import QuantumRegister, QuantumCircuit

import pandas as pd, numpy as np, pennylane as qml, tensorflow as tf
from tensorflow.keras.layers import Lambda, Dropout
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import roc_curve, auc

from pennylane.templates import AmplitudeEmbedding, StronglyEntanglingLayers

# ---  1. Data Preparation ---
tf.keras.backend.set_floatx("float32")
tf.config.run_functions_eagerly(True)

# --- 2. Hyperparameters for model setup ---
CSV_PATH   = "C:\\Users\\fenlei\\Downloads\\machinelearning_data_EEG.csv"
N_QUBITS   = 2                # 2**4 = 16 amplitudes
AMP_DIM    = 2 ** N_QUBITS
N_LAYERS   = 3
BATCH_SIZE = 32
EPOCHS     = 20
VAL_SPLIT  = 0.1

# ---  3. Classical Processing ---
df = pd.read_csv(CSV_PATH).dropna(axis=1, how="all")
# Create a custom mapping
label_mapping = {'Depressive disorder': 1, 'Healthy control': 0}

# Assuming your DataFrame is 'df' and the target column is the first object type column
target = df.select_dtypes(include="object").columns[0]

# Map the labels explicitly
y = df[target].map(label_mapping).astype("float32")

X = df.drop(columns=[target]).select_dtypes("number")
X = SimpleImputer(strategy="mean").fit_transform(X)
X = StandardScaler().fit_transform(X).astype("float32")
X = PCA(n_components=AMP_DIM).fit_transform(X).astype("float32")   # 16 feats
X = normalize(X)
X_tr, X_test, y_tr, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=42
)

print('Processing done')
# --- 4. Quantum feature map and kernel definitions ---
num_qubits = 2  # 2^2 = 4 amplitudes
def feature_map_circuit(x):
    qr = QuantumRegister(num_qubits)
    qc = QuantumCircuit(qr)

    # Amplitude embedding
    qc.append(Initialize(x, normalize=True), qr)

    # Phase encoding via DiagonalGate
    phases = [xi % (2 * np.pi) for xi in x]
    diag = [np.cos(p) + 1j * np.sin(p) for p in phases]
    qc.append(DiagonalGate(diag), qr)

    # return transpile(qc, basis_gates=['u3', 'cx'])
    return qc

def quantum_kernel(x1, x2):
    sv1 = Statevector.from_instruction(feature_map_circuit(x1))
    sv2 = Statevector.from_instruction(feature_map_circuit(x2))
    return np.abs(np.dot(np.conjugate(sv1.data), sv2.data))**2

def build_kernel_matrix(A, B):
    return np.array([[quantum_kernel(a, b) for b in B] for a in A])

# --- 5. Build kernels and train SVM ---
K_train = build_kernel_matrix(X_train, X_train)
clf = SVC(kernel='precomputed', probability=True, class_weight = 'balanced')
clf.fit(K_train, y_train)

K_test = build_kernel_matrix(X_test, X_train)
y_pred = clf.predict(K_test)

# --- 6. Evaluate Result ---
print("Accuracy on test set:", accuracy_score(y_test, y_pred))
from sklearn.metrics import confusion_matrix, classification_report

cm = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:\n", cm)
print(classification_report(y_test, y_pred))
import pickle
with open('qsvm.pkl', 'wb') as f:
    pickle.dump(clf, f)


Processing done
Accuracy on test set: 0.6949152542372882
Confusion Matrix:
 [[15  4]
 [14 26]]
              precision    recall  f1-score   support

         0.0       0.52      0.79      0.62        19
         1.0       0.87      0.65      0.74        40

    accuracy                           0.69        59
   macro avg       0.69      0.72      0.68        59
weighted avg       0.75      0.69      0.70        59

